# Client Partner Implementation Outline


In [17]:
# %pip install owslib
%pip install git+https://github.com/pduchesne/OWSLib.git

import re
import folium
import json, requests, xml
import pandas as pd
import ipywidgets as widgets

from xml.dom import minidom
from IPython.display import Image
from xml.etree import ElementTree
from IPython.display import HTML
from IPython.display import Markdown as md

# Verification of SSL certificate is to be set to False for the eocat endpoint to work.
# verify_ssl = False
verify_ssl = True

  Cloning https://github.com/pduchesne/OWSLib.git to c:\users\yce\appdata\local\temp\pip-req-build-k9dti0sv
  Resolved https://github.com/pduchesne/OWSLib.git to commit 3eb52c8b04c1b74e4326f4291b965ab3f866f571
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/pduchesne/OWSLib.git 'C:\Users\yce\AppData\Local\Temp\pip-req-build-k9dti0sv'

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
#:tags: [remove-cell]
def load_dataframe( resp ):
  
  df = pd.DataFrame(columns=['dc:identifier', 'atom:title', 'atom:updated', 'atom:link[rel="search"]', 'atom:link[rel="enclosure"]', 'atom:link[rel="icon"]'])

  rt = ElementTree.fromstring(response.text)
  for r in rt.findall('{http://www.w3.org/2005/Atom}entry'):
     name = r.find('{http://purl.org/dc/elements/1.1/}identifier').text
     title = r.find('{http://www.w3.org/2005/Atom}title').text
     updated = r.find('{http://www.w3.org/2005/Atom}updated').text
     dcdate = r.find('{http://purl.org/dc/elements/1.1/}date').text
     # print('collection',count,'-', name, ':')
     # print('\tidentifier: ',name)
     
     try:
         href = r.find('{http://www.w3.org/2005/Atom}link[@rel="search"][@type="application/opensearchdescription+xml"]').attrib['href']
     except AttributeError:
         href= ''

     try:
         rel_enclosure = r.find('{http://www.w3.org/2005/Atom}link[@rel="enclosure"]').attrib['href']
     except AttributeError:
         rel_enclosure= ''

     try:
         rel_icon = r.find('{http://www.w3.org/2005/Atom}link[@rel="icon"]').attrib['href']
     except AttributeError:
         rel_icon= ''

     # append a row to the df 
     new_row = { 'dc:identifier': name, 'atom:title': title, 'dc:date': dcdate, 'atom:updated': updated, 'atom:link[rel="search"]': href, 
        'atom:link[rel="enclosure"]': rel_enclosure , 'atom:link[rel="icon"]': rel_icon}
     # df = df.append(new_row, ignore_index=True) 
     dfn = pd.DataFrame(new_row, index = [0])
     df = pd.concat([df, dfn], ignore_index=True)

  return df

def load_facet( root, facet ):
  # create dataframe with information about 'facet' found in XML tree with 'root".
  ns = {'sru': 'http://a9.com/-/opensearch/extensions/sru/2.0/'}
  el = root.find('.//sru:facet[sru:index="' + facet + '"]', ns)
  
  df = pd.DataFrame(columns=['name', 'count'])

  for r in el.findall('.//sru:term', ns):
    name = r.find('sru:actualTerm', ns).text
    count = r.find('sru:count', ns).text
    
    # append a row to the df 
    dfn = pd.DataFrame({ 'name': name, 'count': int(count) }, index = [0])
    df = pd.concat([df, dfn], ignore_index=True)

  df.set_index('name', inplace=True)
  return df


def show_features_on_map( georss_box, georss_polygon ):
  # display map showing the 'polygon' and use center of 'box' to center the map.
  list1 = georss_polygon.split()
  list2 = georss_box.split()
  
  center = [50.85, 4.3488]
  
  if (georss_box == '' or georss_polygon=='' ):
      
    m = folium.Map(location=center,zoom_start=5)
    folium.LayerControl().add_to(m)
      
  else:
      
    points = []    
    for i in range(0,len(list1),2):
        # print (list2[i], list2[i+1])
        points = points + [ (float(list1[i]), float(list1[i+1])) ]
      
    # use center of the bounding box.
    try:
         center = [ (float(list2[0])+float(list2[2]))/2.0 , (float(list2[1])+float(list2[3]))/2.0 ]  
    except:
         # default center and zoom factor for an empty map
         zoom = 3
         
    # m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center)
    m = folium.Map(location=center,zoom_start=2)
    # polygon = Polygon(locations=points, color="green", fill_color="green")
    folium.Polygon(points).add_to(m)
    folium.FitBounds(points, padding=(100,100)).add_to(m)
    # m.add_layer(polygon)
    folium.LayerControl().add_to(m)
    # m.add_control(FullScreenControl())
    # m.add_control(LayersControl(position='topright'))
  
  return m

In [19]:
def get_api_request(template, os_querystring):
  # Fill (URL) template with OpenSearch parameter values provided in os_querystring and return as short HTTP URL without empty parameters.
  
  print("URL template: " + template)
  
  # Limitation: the OSDD may use a default namespace for OpenSearch instead of using "os".
  # We make a simple correction here allowing to use OpenSearch queryables without namespace in requests.
  # A more generic solution to obtain namespaces from the OSDD and compare them with user supplied namespaces is future work.
  
  OS_NAMESPACE = 'os:'
      
  # perform substitutions in template
  for p in os_querystring:
      # print("  .. replacing:", p, "by", os_querystring[p])
      # template = re.sub('\{'+p+'.*?\}', os_querystring[p] , template)
      result = re.subn('\{'+p+'.*?\}', os_querystring[p] , template)
      n = result[1]
      template = result[0]
      if (n<1):
          if (':' in p):
                print("ERROR: parameter " + p + " not found in template.")
          else:
                # try with explicit namespace
                result = re.subn('\{'+OS_NAMESPACE+p+'.*?\}', os_querystring[p] , template)
                n = result[1]
                template = result[0]
                if (n<1):
                    print("ERROR: parameter " + OS_NAMESPACE+p + " not found in template.")   
      
      # print("- intermediate new template:" + template)
      
  # remove empty search parameters
  template=re.sub('&?[a-zA-Z]*=\{.*?\}', '' , template)
  
  # remove remaining empty search parameters which did not have an HTTP query parameter attached (e.g. /{time:end}).
  template=re.sub('.?\{.*?\}', '' , template)
  
  print("API request: " + template)
            
  return (template)

## Client Search Scenario

All Connected Data Asset clients are, at their core, just conventional OpenSearch clients with a few extensions and conventions to make the CEOS OpenSearch protocol useful in the WGISS environment. The IDN identifier of the collection of interest is a mandatory element in CEOS OpenSearch request. Clients could retrieve the collection ID from the IDN OpenSearch Description Document (OSDD).  

This chapter will give brief steps about how to retrieve an IDN collection ID and how to interact with the granule search server for inventory search. The corresponding details are elaborated in the Use Case chapters. 

Data providers will have registered the metadata of their archived collections into the IDN. The client can query the IDN to retrieve the IDN collection ID for a desired collection of interest and, based on that collection ID and other spatial-temporal query conditions, build a valid CEOS OpenSearch query. The following steps describe the client search scenario starting with IDN OpenSearch. 

**Step 1**  
>  Obtain the IDN OpenSearch OSDD to formulate a valid IDN OpenSearch request.

In [20]:
URL_OSDD = "https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc"

In [21]:
md("The OpenSearch Description Document is accessible at the fixed location [{url}]({url}) and contains the URL template to be used for collection search.".format(url=URL_OSDD))

The OpenSearch Description Document is accessible at the fixed location [https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc](https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc) and contains the URL template to be used for collection search.

In [22]:
response = requests.get( URL_OSDD )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><os:OpenSearchDescription xmlns:atom="http://www.w3.org/2005/Atom" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:params="http://a9.com/-/spec/opensearch/extensions/parameters/1.0/" xmlns:referrer="http://www.opensearch.org/Specifications/OpenSearch/Extensions/Referrer/1.0" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">  
    <os:ShortName>CMR Collections</os:ShortName>  
    <os:Description>NASA CMR Collection search using geo, time and parameter extensions</os:Description>  
    <os:Contact>echodev@echo.nasa.gov</os:Contact>  
    <os:Url params:method="GET" rel="collection" template="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&amp;instrument={echo:instrument?}&amp;satellite={eo:platform?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;hasGranules={echo:hasGranules?}&amp;isCwic={echo:isCwic?}&amp;isGeoss={echo:isGeoss?}&amp;isCeos={echo:isCeos?}&amp;isEosdis={echo:isEosdis?}&amp;isFedeo={echo:isFedeo?}&amp;provider={echo:provider?}&amp;clientId=ceosOpenSearchDoc" type="application/atom+xml">    
        <params:Parameter minimum="0" name="keyword" title="Inventory with terms expressed by these search terms" uiDisplay="Search terms" value="{os:searchTerms}">      
            <atom:link href="http://www.elasticsearch.org/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html" rel="profile" title="This parameter follows the elastic search free text search implementations"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="instrument" title="Inventory associated with a satellite instrument expressed by this short name" uiDisplay="Instrument" value="{echo:instrument}"/>    
        <params:Parameter minimum="0" name="satellite" title="Inventory associated with a Satellite/platform expressed by this short name" uiDisplay="Satellite" value="{eo:platform}"/>    
        <params:Parameter minimum="0" name="boundingBox" title="Inventory with a spatial extent overlapping this bounding box" uiDisplay="Bounding box" value="{geo:box}"/>    
        <params:Parameter maxInclusive="90.0" minInclusive="-90.0" minimum="0" name="lat" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" uiDisplay="Latitude" value="{geo:lat}"/>    
        <params:Parameter maxInclusive="180.0" minInclusive="-180.0" minimum="0" name="lon" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" uiDisplay="Longitude" value="{geo:lon}"/>    
        <params:Parameter maxInclusive="6000000" minInclusive="10" minimum="0" name="radius" title="Inventory with the search radius in meters, must be used together with lat and lon" uiDisplay="Radius" value="{geo:radius}"/>    
        <params:Parameter minimum="0" name="geometry" title="Inventory with a spatial extent overlapping this geometry" uiDisplay="Geometry" value="{geo:geometry}">      
            <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineStrings"/>      
            <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Points"/>      
            <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="placeName" title="Inventory with a spatial location described by this name" uiDisplay="Place name" value="{geo:name}"/>    
        <params:Parameter minimum="0" name="startTime" title="Inventory with a temporal extent containing this start time" uiDisplay="Start time" value="{time:start}"/>    
        <params:Parameter minimum="0" name="endTime" title="Inventory with a temporal extent containing this end time" uiDisplay="End time" value="{time:end}"/>    
        <params:Parameter minimum="0" name="cursor" title="Start page for the search result" uiDisplay="Start page" value="{os:startPage}"/>    
        <params:Parameter maxInclusive="2000" minimum="0" name="numberOfResults" title="Maximum number of records in the search result" uiDisplay="Number of results" value="{os:count}"/>    
        <params:Parameter minimum="0" name="offset" title="0-based offset used to skip the specified number of results in the search result set" uiDisplay="Start index" value="{os:startIndex}"/>    
        <params:Parameter minimum="0" name="uid" title="Inventory associated with this unique ID" uiDisplay="Unique identifier" value="{geo:uid}"/>    
        <params:Parameter name="hasGranules" title="Inventory with granules" uiDisplay="Has granules" value="{echo:hasGranules}">      
            <params:Option label="Yes" value="true"/>      
            <params:Option label="No" value="false"/>      
        </params:Parameter>    
        <params:Parameter name="isCwic" title="Inventory related to CWIC" uiDisplay="CWIC collection" value="{echo:isCwic}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isGeoss" title="Inventory related to GEOSS" uiDisplay="GEOSS collection" value="{echo:isGeoss}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isCeos" title="Inventory related to CEOS" uiDisplay="CEOS collection" value="{echo:isCeos}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isEosdis" title="Inventory related to EOSDIS" uiDisplay="EOSDIS collection" value="{echo:isEosdis}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isFedeo" title="Inventory related to FedEO" uiDisplay="FedEO collection" value="{echo:isFedeo}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="provider" title="Inventory associated with a provider" uiDisplay="Provider" value="{echo:provider}"/>    
        <params:Parameter minimum="0" name="clientId" title="Client identifier to be used for metrics" uiDisplay="Client identifier" value="{referrer:source}"/>    
    </os:Url>  
    <os:Url params:method="GET" rel="collection" template="https://cmr.earthdata.nasa.gov/opensearch/collections.html?keyword={os:searchTerms?}&amp;instrument={echo:instrument?}&amp;satellite={eo:platform?}&amp;boundingBox={geo:box?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;hasGranules={echo:hasGranules?}&amp;isCwic={echo:isCwic?}&amp;isGeoss={echo:isGeoss?}&amp;isCeos={echo:isCeos?}&amp;isEosdis={echo:isEosdis?}&amp;isFedeo={echo:isFedeo?}&amp;provider={echo:provider?}&amp;clientId=ceosOpenSearchDoc" type="text/html">    
        <params:Parameter minimum="0" name="keyword" title="Inventory with terms expressed by these search terms" uiDisplay="Search terms" value="{os:searchTerms}">      
            <atom:link href="http://www.elasticsearch.org/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html" rel="profile" title="This parameter follows the elastic search free text search implementations"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="instrument" title="Inventory associated with a satellite instrument expressed by this short name" uiDisplay="Instrument" value="{echo:instrument}"/>    
        <params:Parameter minimum="0" name="satellite" title="Inventory associated with a Satellite/platform expressed by this short name" uiDisplay="Satellite" value="{eo:platform}"/>    
        <params:Parameter minimum="0" name="boundingBox" title="Inventory with a spatial extent overlapping this bounding box" uiDisplay="Bounding box" value="{geo:box}"/>    
        <params:Parameter maxInclusive="90.0" minInclusive="-90.0" minimum="0" name="lat" title="Inventory with latitude in decimal degrees, must be used together with lon and radius" uiDisplay="Latitude" value="{geo:lat}"/>    
        <params:Parameter maxInclusive="180.0" minInclusive="-180.0" minimum="0" name="lon" title="Inventory with longitude in decimal degrees, must be used together with lat and radius" uiDisplay="Longitude" value="{geo:lon}"/>    
        <params:Parameter maxInclusive="6000000" minInclusive="10" minimum="0" name="radius" title="Inventory with the search radius in meters, must be used together with lat and lon" uiDisplay="Radius" value="{geo:radius}"/>    
        <params:Parameter minimum="0" name="geometry" title="Inventory with a spatial extent overlapping this geometry" uiDisplay="Geometry" value="{geo:geometry}">      
            <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineStrings"/>      
            <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Points"/>      
            <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="placeName" title="Inventory with a spatial location described by this name" uiDisplay="Place name" value="{geo:name}"/>    
        <params:Parameter minimum="0" name="startTime" title="Inventory with a temporal extent containing this start time" uiDisplay="Start time" value="{time:start}"/>    
        <params:Parameter minimum="0" name="endTime" title="Inventory with a temporal extent containing this end time" uiDisplay="End time" value="{time:end}"/>    
        <params:Parameter minimum="0" name="cursor" title="Start page for the search result" uiDisplay="Start page" value="{os:startPage}"/>    
        <params:Parameter maxInclusive="2000" minimum="0" name="numberOfResults" title="Maximum number of records in the search result" uiDisplay="Number of results" value="{os:count}"/>    
        <params:Parameter minimum="0" name="offset" title="0-based offset used to skip the specified number of results in the search result set" uiDisplay="Start index" value="{os:startIndex}"/>    
        <params:Parameter minimum="0" name="uid" title="Inventory associated with this unique ID" uiDisplay="Unique identifier" value="{geo:uid}"/>    
        <params:Parameter name="hasGranules" title="Inventory with granules" uiDisplay="Has granules" value="{echo:hasGranules}">      
            <params:Option label="Yes" value="true"/>      
            <params:Option label="No" value="false"/>      
        </params:Parameter>    
        <params:Parameter name="isCwic" title="Inventory related to CWIC" uiDisplay="CWIC collection" value="{echo:isCwic}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isGeoss" title="Inventory related to GEOSS" uiDisplay="GEOSS collection" value="{echo:isGeoss}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isCeos" title="Inventory related to CEOS" uiDisplay="CEOS collection" value="{echo:isCeos}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isEosdis" title="Inventory related to EOSDIS" uiDisplay="EOSDIS collection" value="{echo:isEosdis}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter name="isFedeo" title="Inventory related to FedEO" uiDisplay="FedEO collection" value="{echo:isFedeo}">      
            <params:Option label="Yes" value="true"/>      
        </params:Parameter>    
        <params:Parameter minimum="0" name="provider" title="Inventory associated with a provider" uiDisplay="Provider" value="{echo:provider}"/>    
        <params:Parameter minimum="0" name="clientId" title="Client identifier to be used for metrics" uiDisplay="Client identifier" value="{referrer:source}"/>    
    </os:Url>  
    <os:Query role="example" searchTerms="Amazon River Basin Precipitation, 1972-1992" title="Sample search"/>  
    <os:Attribution>NASA CMR</os:Attribution>  
    <os:SyndicationRight>open</os:SyndicationRight>  
    <os:Tags>CMR NASA CWIC CEOS-OS-BP-V1.1/L3 ESIP OGC collection pageOffset=1 indexOffset=0</os:Tags>  
</os:OpenSearchDescription>
```


The OSDD may contain URL templates for multiple values for `type` (media types) and multiple values for `rel` (relations).  The `rel` values have the following meaning (if present) as defined in [[RD1]](#RD1). :

| **rel** |  **description**  | 
| ----- | ----- |
| collection |  URL template to be used for collection search.  |
| results |  URL template to be used for granule search (default).  |
| service |  URL template to be used for service or application search.  |

The current OSDD contains the following URL templates:

In [23]:
from xml.etree import ElementTree
root = ElementTree.fromstring(response.text)

list = pd.DataFrame(columns=['rel', 'type'])
ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
for r in root.findall('os:Url', ns):  
    rel = ''
    mtype = ''
    try:
        rel   = r.attrib['rel']
        mtype = r.attrib['type']
    except:
        pass
    df = pd.DataFrame({ 'rel': rel, 'type': mtype }, index = [0])
    list = pd.concat([list, df], ignore_index=True)  
  
list     

,rel,type
0,collection,application/atom+xml
1,collection,text/html


In [24]:
# find URL template for collection search
from xml.etree import ElementTree
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
collection_url_atom = root.find('os:Url[@rel="collection"][@type="application/atom+xml"]', ns)

collection_template = collection_url_atom.attrib['template']
collection_template

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&instrument={echo:instrument?}&satellite={eo:platform?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&hasGranules={echo:hasGranules?}&isCwic={echo:isCwic?}&isGeoss={echo:isGeoss?}&isCeos={echo:isCeos?}&isEosdis={echo:isEosdis?}&isFedeo={echo:isFedeo?}&provider={echo:provider?}&clientId=ceosOpenSearchDoc'

**Step 2**  
>  Search collections of interest through IDN OpenSearch with proper request parameters (e.g. spatial footprint, temporal extent and keyword). 

In [25]:
# test
# https://owslib.readthedocs.io/en/latest/usage.html#opensearch
# https://github.com/geopython/OWSLib/blob/master/tests/test_opensearch_creodias.py
# https://github.com/geopython/OWSLib/blob/master/owslib/opensearch.py


# from owslib.opensearch import OpenSearch
# o = OpenSearch(URL_OSDD)

# o.description.urls

# print(o.description.urls['application/atom+xml']['parameters'])

# assert o.description.shortname == 'CMR Collections'
# assert o.description.description == 'NASA CMR Collection search using geo, time and parameter extensions'

# searches do not work.  Not with HTTP query paremeters and not with OpenSearch parameters with namespace in front (with underscore).
# results = o.search('application/atom+xml', eo_platform='landsat')  # dict of results
# results = o.search('application/atom+xml', satellite='landsat') 
# results = o.search('application/atom+xml', keyword = 'landsat')  # dict of results


In [26]:
osquerystring = {'count': '10', 'searchTerms': 'Landsat*'}
request_url = get_api_request(collection_template, osquerystring)

response = requests.get( request_url )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
md("```xml\n" + xmlstr + "\n```\n")

URL template: https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&instrument={echo:instrument?}&satellite={eo:platform?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&hasGranules={echo:hasGranules?}&isCwic={echo:isCwic?}&isGeoss={echo:isGeoss?}&isCeos={echo:isCeos?}&isEosdis={echo:isEosdis?}&isFedeo={echo:isFedeo?}&provider={echo:provider?}&clientId=ceosOpenSearchDoc
API request: https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&numberOfResults=10&clientId=ceosOpenSearchDoc


```xml
<?xml version="1.0" ?><feed esipdiscovery:version="1.2" xmlns="http://www.w3.org/2005/Atom" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:esipdiscovery="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:georss="http://www.georss.org/georss" xmlns:gml="http://www.opengis.net/gml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:relevance="http://a9.com/-/opensearch/extensions/relevance/1.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">   
     <updated>2024-10-11T15:01:34.373Z</updated>   
     <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom</id>   
     <author>      
          <name>CMR</name>      
          <email>echodev@echo.nasa.gov</email>      
     </author>   
     <title type="text">ECHO dataset metadata</title>   
     <subtitle type="text">Search parameters: keyword =&gt; Landsat*</subtitle>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml" hreflang="en-US" rel="search" type="application/opensearchdescription+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="self" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=66" hreflang="en-US" rel="last" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=2" hreflang="en-US" rel="next" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=Landsat*&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="first" type="application/atom+xml"/>   
     <link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" rel="describedBy" title="Release Notes" type="text/html"/>   
     <os:Query os:searchTerms="Landsat*" role="request" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/"/>   
     <os:totalResults>654</os:totalResults>   
     <os:itemsPerPage>10</os:itemsPerPage>   
     <os:startIndex>1</os:startIndex>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1965336923-ESA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>FEDEO</consortium>      
          <title type="text">Landsat 8 OLI-TIRS European Coverage</title>      
          <summary type="text">The European coverage of Landsat 8 data since the beginning of the mission until December 2014 is available on the Landsat 8 portal. Data shown on the portal: portal are reprocessed (Tier 1 and Tier 2) products acquired at ESA receiving stations (Neustrelitz, Matera and Kiruna visibility masks).: You can find the most recent data (from 2015 onwards) in the Landsat8 European Coverage Collection 2 Level 1 and Level2; as soon as the new collection is completely uploaded and the entire mission is covered, the former dissemination system for Collection 1 dataset will be dismissed. Landsat 8 level 1 products combine data from the two Landsat instruments, OLI and TIRS. The level 1 products generated can be either L1T or L1gT (30m resolution, 170 km x 185 km scene size): • L1T - Orthorectified products Level 1T data products consist of radiometrically corrected image data derived from L0 data scaled to at-aperture spectral radiance or reflectance that are resampled for registration to a cartographic projection (referenced to the WGS84, G873 or current version). The L1T product is orthorectified, and corrected for terrain relief. The geometric corrections use observatory ephemeris data and ground control points; DEM data is used to correct for terrain relief. • L1gT - Geometrically corrected products L1gT data products consist of L0 product data with systematic radiometric, geometric and terrain corrections applied and resampled for registration to a cartographic projection, referenced to the WGS84, G873, or current version. L1gT data products assume the use of on-board positional information or definitive ephemeris, as well as the use of controlled elevation data to correct for parallax errors. In case of insufficient GCP, caused by extensive snow/cloud cover, which makes the orthorectification impossible, products of this level are produced instead of L1T.</summary>      
          <updated/>      
          <link href="https://fedeo-client.ceos.org?parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=LANDSAT.OLI-TIRS.L1T_L1GT" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/opensearch/request/?httpAccept=application/vnd.iso.19139-2%2Bxml&amp;parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=LANDSAT.OLI-TIRS.L1T_L1GT&amp;recordSchema=iso19139-2" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT.OLI-TIRS.L1T_L1GT_NA" hreflang="en-US" rel="enclosure" title="LANDSAT.OLI-TIRS.L1T_L1GT" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1965336923-ESA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1965336923-ESA</dc:identifier>      
          <dc:date>2014-12-31T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat 8 OLI-TIRS European Coverage</echo:datasetId>      
          <echo:shortName>LANDSAT.OLI-TIRS.L1T_L1GT</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:dataCenter>ESA</echo:dataCenter>      
          <echo:organization>ESA/ESRIN</echo:organization>      
          <echo:processingLevelId>1</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-35 -20 75 35</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine/>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>int.esa.fedeo</echo:tagKey>         
          </echo:tag>      
          <echo:is_fedeo>true</echo:is_fedeo>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C2241716601-ESA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>FEDEO</consortium>      
          <title type="text">Landsat 8 Collection 2 European Coverage</title>      
          <summary type="text">This dataset contains the European Coverage of Landsat 8 Collection 2 data, both Level 1 and Level 2, since the beginning of the mission. Landsat 8 Collection 2 is the result of reprocessing effort on the archive and on fresh products with significant improvement with respect to Collection 1 on data quality, obtained by means of advancements in data processing, algorithm development. The primary characteristic is a relevant improvement in the absolute geolocation accuracy (now re-baselined to the European Space Agency Copernicus Sentinel-2 Global Reference Image, GRI) but includes also updated digital elevation modelling sources, improved Radiometric Calibration (even correction for the TIRS striping effect), enhanced Quality Assessment Bands, updated and consistent metadata files, usage of Cloud Optimized Georeferenced (COG) Tagged Image File Format.  Landsat 8 level 1 products combine data from the 2 Landsat instruments, OLI and TIRS. The level 1 products generated can be either L1TP or L1GT: • L1TP - Level 1 Precision Terrain (Corrected) (L1T) products: Radiometrically calibrated and orthorectified using ground control points (GCPs) and digital elevation model (DEM) data to correct for relief displacement. The highest quality Level-1 products suitable for pixel-level time series analysis. GCPs used for L1TP correction are derived from the Global Land Survey 2000 (GLS2000) data set. • L1GT - Level 1 Systematic Terrain (Corrected) (L1GT) products: L1GT data products consist of L0 product data with systematic radiometric, geometric and terrain corrections applied and resampled for registration to a cartographic projection, referenced to the WGS84, G873, or current version. The dissemination server contains three different classes of Level1 products • Real Time (RT): Newly acquired Landsat 8 OLI/TIRS data are processed upon downlink but use an initial TIRS line-of-sight model parameters; the data is made available in less than 12 hours (4-6 hours typically). Once the data have been reprocessed with the refined TIRS parameters, the products are transitioned to either Tier 1 or Tier 2 and removed from the Real-Time tier (in 14-16 days). • Tier 1 (T1): Landsat scenes with the highest available data quality are placed into Tier 1 and are considered suitable for time-series analysis. Tier 1 includes Level-1 Precision and Terrain (L1TP) corrected data that have well-characterized radiometry and are inter-calibrated across the different Landsat instruments. The georegistration of Tier 1 scenes is consistent and within prescribed image-to-image tolerances of ≦ 12-meter radial root mean square error (RMSE). • Tier 2 (T2): Landsat scenes not meeting Tier 1 criteria during processing are assigned to Tier 2. Tier 2 scenes adhere to the same radiometric standard as Tier 1 scenes, but do not meet the Tier 1 geometry specification due to less accurate orbital information (specific to older Landsat sensors), significant cloud cover, insufficient ground control, or other factors. This includes Systematic Terrain (L1GT) and Systematic (L1GS) processed data.  Landsat 8 level 2 products are generated from L1GT and L1TP Level 1 products that meet the &lt;76 degrees Solar Zenith Angle constraint and include the required auxiliary data inputs to generate a scientifically viable product. The data are available a couple of days after the Level1 T1/T2. The level 2 products generated can be L2SP or L2SR: • L2SP - Level 2 Science Products (L2SP) products: include Surface Reflectance (SR), Surface Temperature (ST), ST intermediate bands, an angle coefficients file, and Quality Assessment (QA) Bands. • L2SR - Level 2 Surface Reflectance (L2SR) products: include Surface Reflectance (SR), an angle coefficients file, and Quality Assessment (QA) Bands; it is generated if ST could not be generated  Two different categories of Level 1 products are offered: LC with Optical, Thermal and Quality Map images, LO with Optical and Quality Map images (Thermal not available). For the Level 2 data, only LC combined products are generated</summary>      
          <updated/>      
          <link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=Landsat8.Collection2.European.Coverage" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://fedeo-client.ceos.org?parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=Landsat8.Collection2.European.Coverage" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/opensearch/request/?httpAccept=application/vnd.iso.19139-2%2Bxml&amp;parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=Landsat8.Collection2.European.Coverage&amp;recordSchema=iso19139-2" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?Landsat8.Collection2.European.Coverage_NA" hreflang="en-US" rel="enclosure" title="Landsat8.Collection2.European.Coverage" type="text/html"/>      
          <link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=Landsat8.Collection2.European.Coverage" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C2241716601-ESA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C2241716601-ESA</dc:identifier>      
          <dc:date>2015-01-01T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat 8 Collection 2 European Coverage</echo:datasetId>      
          <echo:shortName>Landsat8.Collection2.European.Coverage</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:dataCenter>ESA</echo:dataCenter>      
          <echo:organization>ESA/ESRIN</echo:organization>      
          <echo:processingLevelId>1</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-35 -20 75 35</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1214584349-SCIOPS</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <title type="text">LANDSAT MSS Data Received at the NASDA Station in Japan</title>      
          <summary type="text">The LANDSAT Multispectral Scanner (MSS), a mechanical scanning
      radiometer, operates in 4 channels of the electro-magnetic spectrum,
      from 0.55 to 1.00 micrometers.
      
      Data collected by the Earth Observation Center, NASDA, cover a
      circular area about 5000 km diameter.  The Earth Observation Center
      receives MSS data weekdays and every other Saturday. It performs both
      radiometric and geometric correction and distributes products in the
      form of magnetic tape and imagery.</summary>      
          <updated/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT-MSS-NASDA" hreflang="en-US" rel="enclosure" title="LANDSAT-MSS-NASDA" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1214584349-SCIOPS.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1214584349-SCIOPS</dc:identifier>      
          <dc:date>1979-01-29T00:00:00.000Z/</dc:date>      
          <echo:datasetId>LANDSAT MSS Data Received at the NASDA Station in Japan</echo:datasetId>      
          <echo:shortName>LANDSAT-MSS-NASDA</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:dataCenter>SCIOPS</echo:dataCenter>      
          <echo:archiveCenter>JP/RESTEC</echo:archiveCenter>      
          <echo:organization>JP/RESTEC</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>23 115 55 153</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1214584333-SCIOPS</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <title type="text">LANDSAT Thematic Mapper Data Received at the NASDA Station in Japan</title>      
          <summary type="text">The LANDSAT Thematic Mapper, a mechanical scanning radiometer, operates in 7
channels of electro-magnetic spectra, including visual, near infrared and
thermal infrared.

Data collected by the Earth Observation Center, NASDA, cover a circular area
about 5000 km diameter.  The Earth Observation Center receives TM data weekdays
and every other Saturday. It performs both radiometric and geometric correction
and distributes products in the form of magnetic tape and imagery.</summary>      
          <updated/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT-TM-NASDA" hreflang="en-US" rel="enclosure" title="LANDSAT-TM-NASDA" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1214584333-SCIOPS.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1214584333-SCIOPS</dc:identifier>      
          <dc:date>1984-03-24T00:00:00.000Z/</dc:date>      
          <echo:datasetId>LANDSAT Thematic Mapper Data Received at the NASDA Station in Japan</echo:datasetId>      
          <echo:shortName>LANDSAT-TM-NASDA</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:dataCenter>SCIOPS</echo:dataCenter>      
          <echo:archiveCenter>JP/RESTEC</echo:archiveCenter>      
          <echo:organization>JP/RESTEC</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>23 115 55 153</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1220567985-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">LANDSAT SURFACE REFLECTANCE -L7 ETM+</title>      
          <summary type="text">Landsat satellite data have been produced, archived, and distributed by the U.S. Geological Survey (USGS) since 1972.  Users rely on these data for historical study of land surface change but shoulder the burden of post-production processing to create applications-ready data sets.</summary>      
          <updated/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT_SURFACE_REFLECTANCE_L7_ETM" hreflang="en-US" rel="enclosure" title="LANDSAT_SURFACE_REFLECTANCE_L7_ETM" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1220567985-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1220567985-USGS_LTA</dc:identifier>      
          <dc:date>2013-02-13T00:00:00.000Z/</dc:date>      
          <echo:datasetId>LANDSAT SURFACE REFLECTANCE -L7 ETM+</echo:datasetId>      
          <echo:shortName>LANDSAT_SURFACE_REFLECTANCE_L7_ETM</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-90 -180 90 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1220567969-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">Landsat Surface Reflectance - L8 OLI/TIRS</title>      
          <summary type="text">Landsat satellite data have been produced, archived, and distributed by the U.S. Geological Survey (USGS) since 1972.  Users rely on these data for historical study of land surface change but shoulder the burden of post-production processing to create applications-ready data sets.</summary>      
          <updated/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT_SURFACE_REFLECTANCE_L8_OLI_TIRS" hreflang="en-US" rel="enclosure" title="LANDSAT_SURFACE_REFLECTANCE_L8_OLI_TIRS" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1220567969-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1220567969-USGS_LTA</dc:identifier>      
          <dc:date>2013-02-01T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat Surface Reflectance - L8 OLI/TIRS</echo:datasetId>      
          <echo:shortName>LANDSAT_SURFACE_REFLECTANCE_L8_OLI_TIRS</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-90 -180 90 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1235542031-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">Landsat 8</title>      
          <summary type="text">The Operational Land Imager (OLI) and Thermal Infrared Sensor (TIRS) are onboard the Landsat 8 satellite, have acquired images of the Earth since  February 2013. The sensors collect images of the Earth with a 16-day repeat cycle, referenced to the Worldwide Reference System-2. The approximate scene size is 170 km north-south by 183 km east-west (106 mi by 114 mi).

Landsat 8 image data files consist of 11 spectral bands with a spatial resolution of 30 meters for bands 1-7 and bands 9-11; 15-meters for the panchromatic band 8. Delivered Landsat 8 Level-1 data typically include both OLI and TIRS data files; however, there may be OLI-only and/or TIRS-only scenes in the USGS archive.  A Quality Assurance (QA.tif) band is also included. This file provides bit information regarding conditions that may affect the accuracy and usability of a given pixel – clouds, water or snow, for example.</summary>      
          <updated/>      
          <link href="http://glovis.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://landsat.usgs.gov/landsat8.php" hreflang="en-US" rel="describedBy"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?collectionConceptId=C1235542031-USGS_LTA" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1235542031-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1235542031-USGS_LTA</dc:identifier>      
          <dc:date>2013-02-11T00:00:00.000Z/</dc:date>      
          <echo:datasetId>Landsat 8</echo:datasetId>      
          <echo:shortName>Landsat_8</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:processingLevelId>Not Provided</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-82.71 -180 82.74 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>org.ceos.wgiss.cwic.granules.prod</echo:tagKey>         
               <echo:data>&quot; &quot;</echo:data>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.ceos.wgiss.cwic.granules.provider</echo:tagKey>         
               <echo:data>&quot;USGSLSI&quot;</echo:data>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C3108204618-INPE</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <title type="text">Landsat Collection 2 - Level-2</title>      
          <summary type="text">Landsat Collection 2 Level-2 Science Products (https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products), consisting of atmospherically corrected surface reflectance (https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-reflectance) and surface temperature (https://www.usgs.gov/landsat-missions/landsat-collection-2-surface-temperature) image data. Collection 2 Level-2 Science Products are available from August 22, 1982 to present. This dataset represents the Brazilian archive of Level-2 data from Landsat Collection 2 (https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2) acquired by the Thematic Mapper (https://landsat.gsfc.nasa.gov/thematic-mapper/) onboard Landsat 4 and 5, the Enhanced Thematic Mapper (https://landsat.gsfc.nasa.gov/the-enhanced-thematic-mapper-plus-etm/) onboard Landsat 7, and Operatational Land Imager (https://landsat.gsfc.nasa.gov/satellites/landsat-8/spacecraft-instruments/operational-land-imager/) and Thermal Infrared Sensor (https://landsat.gsfc.nasa.gov/satellites/landsat-8/spacecraft-instruments/thermal-infrared-sensor/) onboard Landsat 8 and 9. Images are stored in cloud-optimized GeoTIFF (https://www.cogeo.org/) format.</summary>      
          <updated/>      
          <link href="https://fedeo-client.ceos.org?uid=landsat-2" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/landsat-2?httpAccept=application/vnd.iso.19139-2%2Bxml" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/landsat-2/api" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?landsat-2_NA" hreflang="en-US" rel="enclosure" title="landsat-2" type="text/html"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/landsat-2/api" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C3108204618-INPE.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C3108204618-INPE</dc:identifier>      
          <dc:date>2008-01-01T00:00:00.000Z/2024-06-14T00:00:00.000Z</dc:date>      
          <echo:datasetId>Landsat Collection 2 - Level-2</echo:datasetId>      
          <echo:shortName>landsat-2</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:dataCenter>INPE</echo:dataCenter>      
          <echo:organization>DOI/USGS</echo:organization>      
          <echo:organization>BR/INPE</echo:organization>      
          <echo:processingLevelId>NA</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-35.6706264 -75.6664631 6.8339121 -27.6837564</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C3108204500-INPE</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <title type="text">Landsat Collection 2 - Level-2 - Data Cube - LCF 16 days</title>      
          <summary type="text">Earth Observation Data Cube generated from Landsat Level-2 product over Brazil extension. This dataset is provided in Cloud Optimized GeoTIFF (COG) file format. The dataset is processed with 30 meters of spatial resolution, reprojected and cropped to BDC_MD grid Version 2 (BDC_MD V2), considering a temporal compositing function of 16 days using the Least Cloud Cover First (LCF) best pixel approach.</summary>      
          <updated/>      
          <link href="https://fedeo-client.ceos.org?uid=LANDSAT-16D-1" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/LANDSAT-16D-1?httpAccept=application/vnd.iso.19139-2%2Bxml" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/LANDSAT-16D-1/api" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT-16D-1_NA" hreflang="en-US" rel="enclosure" title="LANDSAT-16D-1" type="text/html"/>      
          <link href="https://fedeo.ceos.org/collections/series/items/LANDSAT-16D-1/api" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C3108204500-INPE.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C3108204500-INPE</dc:identifier>      
          <dc:date>2015-01-01T00:00:00.000Z/2023-12-31T00:00:00.000Z</dc:date>      
          <echo:datasetId>Landsat Collection 2 - Level-2 - Data Cube - LCF 16 days</echo:datasetId>      
          <echo:shortName>LANDSAT-16D-1</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:dataCenter>INPE</echo:dataCenter>      
          <echo:organization>BR/INPE</echo:organization>      
          <echo:processingLevelId>NA</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-34.6755646 -75.9138367 5.9260044 -33.3435683</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
     </entry>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1220567986-USGS_LTA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>CWIC</consortium>      
          <title type="text">LANDSAT SURFACE REFLECTANCE L4-5 TM</title>      
          <summary type="text">Landsat satellite data have been produced, archived, and distributed by the U.S. Geological Survey (USGS) since 1972.  Users rely on these data for historical study of land surface change but shoulder the burden of post-production processing to create applications-ready data sets.</summary>      
          <updated/>      
          <link href="http://earthexplorer.usgs.gov" hreflang="en-US" rel="enclosure"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?LANDSAT_SURFACE_REFLECTANCE_L4-5_TM" hreflang="en-US" rel="enclosure" title="LANDSAT_SURFACE_REFLECTANCE_L4-5_TM" type="text/html"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1220567986-USGS_LTA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1220567986-USGS_LTA</dc:identifier>      
          <dc:date>1982-07-01T00:00:00.000Z/2013-01-01T23:59:59.999Z</dc:date>      
          <echo:datasetId>LANDSAT SURFACE REFLECTANCE L4-5 TM</echo:datasetId>      
          <echo:shortName>LANDSAT_SURFACE_REFLECTANCE_L4-5_TM</echo:shortName>      
          <echo:versionId>Not provided</echo:versionId>      
          <echo:dataCenter>USGS_LTA</echo:dataCenter>      
          <echo:archiveCenter>DOI/USGS/EROS</echo:archiveCenter>      
          <echo:organization>DOI/USGS/EROS</echo:organization>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-90 -180 90 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>2.8028</relevance:score>      
          <echo:tag>         
               <echo:tagKey>org.geoss.geoss_data-core</echo:tagKey>         
          </echo:tag>      
          <echo:tag>         
               <echo:tagKey>gov.nasa.eosdis</echo:tagKey>         
          </echo:tag>      
          <echo:is_geoss>true</echo:is_geoss>      
          <echo:is_eosdis>true</echo:is_eosdis>      
     </entry>   
</feed>
```


**Step 3**  
>  With the IDN OpenSearch response, obtain the CWIC or FedEO OSDD endpoint according to CEOS-BP-001 for the collection by parsing the href attribute in `<link rel="search" type=”application/opensearchdescription+xml” >` (CEOS-BP-001). 

GUI for the search is available at https://search.earthdata.nasa.gov/search?portal=idn

In [27]:
from xml.etree import ElementTree

# IDN_COLLECTION = 'C1235542031-USGS_LTA'       # example from original CDA OpenSearch Client Guide - https://search.earthdata.nasa.gov/search?portal=idn&q=C1235542031-USGS_LTA (does not work).  
# IDN_COLLECTION = "C2237824918-ORNL_CLOUD"
IDN_COLLECTION = "C1532648157-ESA"

osquerystring = {'geo:uid': IDN_COLLECTION}
request_url = get_api_request(collection_template, osquerystring)

response = requests.get( request_url )

root = ElementTree.fromstring(response.text)
# Extract <entry> element 
el = root.find('{http://www.w3.org/2005/Atom}entry')
xmltxt = ElementTree.tostring(el, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

URL template: https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&instrument={echo:instrument?}&satellite={eo:platform?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&hasGranules={echo:hasGranules?}&isCwic={echo:isCwic?}&isGeoss={echo:isGeoss?}&isCeos={echo:isCeos?}&isEosdis={echo:isEosdis?}&isFedeo={echo:isFedeo?}&provider={echo:provider?}&clientId=ceosOpenSearchDoc
API request: https://cmr.earthdata.nasa.gov/opensearch/collections.atom?&uid=C1532648157-ESA&clientId=ceosOpenSearchDoc


```xml
<ns0:entry xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.w3.org/2005/Atom" xmlns:ns2="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:ns3="http://www.georss.org/georss">
    <ns0:id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1532648157-ESA</ns0:id>
    <ns0:author>
      <ns0:name>CMR</ns0:name>
      <ns0:email>echodev@echo.nasa.gov</ns0:email>
    </ns0:author>
    <ns0:consortium>CEOS</ns0:consortium>
    <ns0:consortium>FEDEO</ns0:consortium>
    <ns0:title type="text">TropForest- ALOS, GEOSAT-1 &amp; KOMPSAT-2 optical coverages over tropical forests</ns0:title>
    <ns0:summary type="text">The objective of the ESA TropForest project was to create a harmonised geo-database of ready-to-use satellite imagery to support 2010 global forest assessment performed by the Joint Research Centre (JRC) of the European Commission and by the Food and Agriculture Organization (FAO). Assessments for year 2010 were essential for building realistic deforestation benchmark rates at global to regional levels. To reach this objective, the project aimed to create a harmonised ortho-rectified/pre-processed imagery geo-database based on satellite data acquisitions (ALOS AVNIR-2, GEOSAT-1 SLIM6, KOMPSAT-2 MSC) performed during year 2009 and 2010, for the Tropical Latin America (excluding Mexico) and for the Tropical South and Southeast Asia (excluding China), resulting in 1971 sites located at 1 deg x 1 deg geographical lat/long intersections. The project finally delivered 1866 sites (94.7% of target) due to cloud coverages too high for missing sites</ns0:summary>
    <ns0:updated />
    <ns0:link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=TropForest" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#" />
    <ns0:link href="https://fedeo-client.ceos.org?parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=TropForest" hreflang="en-US" length="0.0KB" rel="enclosure" />
    <ns0:link href="https://fedeo.ceos.org/opensearch/request/?httpAccept=application/vnd.iso.19139-2%2Bxml&amp;parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=TropForest&amp;recordSchema=iso19139-2" hreflang="en-US" length="0.0KB" rel="enclosure" />
    <ns0:link href="https://doi.org/10.5270/esa-qoe849q" hreflang="en-US" rel="via" />
    <ns0:link href="http://gcmd.nasa.gov/getdif.htm?TropForest_NA" hreflang="en-US" rel="enclosure" title="TropForest" type="text/html" />
    <ns0:link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=TropForest" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml" />
    <ns0:link href="https://cmr.earthdata.nasa.gov/search/concepts/C1532648157-ESA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml" />
    <dc:identifier>C1532648157-ESA</dc:identifier>
    <dc:date>2009-01-27T00:00:00.000Z/2011-08-09T23:59:59.999Z</dc:date>
    <ns2:datasetId>TropForest- ALOS, GEOSAT-1 &amp; KOMPSAT-2 optical coverages over tropical forests</ns2:datasetId>
    <ns2:shortName>TropForest</ns2:shortName>
    <ns2:versionId>NA</ns2:versionId>
    <ns2:dataCenter>ESA</ns2:dataCenter>
    <ns2:organization>ESA/ESRIN</ns2:organization>
    <ns2:processingLevelId>1</ns2:processingLevelId>
    <ns2:coordinateSystem>CARTESIAN</ns2:coordinateSystem>
    <ns2:orbitParameters />
    <ns3:box>-50 -100 40 160</ns3:box>
    <ns2:hasVariables>false</ns2:hasVariables>
    <ns2:hasFormats>false</ns2:hasFormats>
    <ns2:hasTransforms>false</ns2:hasTransforms>
    <ns2:hasCombine />
    <ns2:hasSpatialSubsetting>false</ns2:hasSpatialSubsetting>
    <ns2:hasTemporalSubsetting>false</ns2:hasTemporalSubsetting>
    <ns2:cloudHosted>false</ns2:cloudHosted>
    <ns2:tag>
      <ns2:tagKey>int.esa.fedeo</ns2:tagKey>
    </ns2:tag>
    <ns2:is_fedeo>true</ns2:is_fedeo>
  </ns0:entry>

```


In [28]:
# Obtain the endpoint for the granule search for this collection.

root = ElementTree.fromstring(response.text)
# Extract <link> element with the OSDD for the granule search
el = root.find('{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}link[@rel="search"][@type="application/opensearchdescription+xml"]')
xmltxt = ElementTree.tostring(el, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")


```xml
<ns0:link xmlns:ns0="http://www.w3.org/2005/Atom" href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=TropForest" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml" />
    
```


In [29]:
# Extract the short name for future use.  Should not be needed according to CEOS OpenSearch Best Practices.  
el = root.find('{http://www.w3.org/2005/Atom}entry/{https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom}shortName')
el.text
short_name = el.text

**Step 4**  
>  Based on the retrieved OSDD, formulate an OpenSearch request for granules belonging to that collection, directed to the relevant server.  

In [30]:
el = root.find('{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}link[@rel="search"][@type="application/opensearchdescription+xml"]')
url_osdd = el.attrib['href']
el.attrib['href']

'https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=TropForest'

In [31]:
# Get the OSDD document
response = requests.get( el.attrib['href'] )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")


```xml
<?xml version="1.0" ?><OpenSearchDescription xmlns="http://a9.com/-/spec/opensearch/1.1/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" xmlns:param="http://a9.com/-/spec/opensearch/extensions/parameters/1.0/" xmlns:semantic="http://a9.com/-/opensearch/extensions/semantic/1.0/" xmlns:sru="http://a9.com/-/opensearch/extensions/sru/2.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/" xmlns:wsse="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-secext-1.0.xsd">  
  <ShortName>FedEO</ShortName>  
  <Description>Provides interoperable access, following ISO/OGC interface guidelines, to Earth Observation metadata.</Description>  
  <Tags>FedEO Clearinghouse, ESA, Earth Observation, Digital Repository, HMA, HMA-S, HMA-SE, CEOS-OS-BP-V1.1/L1.</Tags>  
  <Url rel="self" template="https://fedeo.ceos.org/collections/series/items/TropForest/api?parentIdentifier=TropForest" type="application/opensearchdescription+xml"/>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fogcapi-features-1%2F1.0%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson%3Bprofile%3D%22https%3A%2F%2Fstacspec.org%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson%3Bprofile%3Dhttps%3A%2F%2Fstacspec.org&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/geo+json;profile=https://stacspec.org" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3D%22http%3A%2F%2Fdata.europa.eu%2F930%2F%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3D%22https%3A%2F%2Fschema.org%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=&quot;https://schema.org&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3Dhttp%3A%2F%2Fdata.europa.eu%2F930%2F&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=http://data.europa.eu/930/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3Dhttp%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=http://www.opengis.net/spec/os-geojson/1.0" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3Dhttps%3A%2F%2Fschema.org&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=https://schema.org" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3D%22http%3A%2F%2Fdata.europa.eu%2F930%2F%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3D%22https%3A%2F%2Fschema.org%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3Dhttp%3A%2F%2Fdata.europa.eu%2F930%2F&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=http://data.europa.eu/930/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3Dhttp%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=http://www.opengis.net/spec/os-geojson/1.0" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3Dhttps%3A%2F%2Fschema.org&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=https://schema.org" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fhtml&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/html" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3D%22http%3A%2F%2Fdata.europa.eu%2F930%2F%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3D%22https%3A%2F%2Fschema.org%22&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=&quot;https://schema.org&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3Dhttp%3A%2F%2Fdata.europa.eu%2F930%2F&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=http://data.europa.eu/930/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3Dhttp%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=http://www.opengis.net/spec/os-geojson/1.0" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3Dhttps%3A%2F%2Fschema.org&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=https://schema.org" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;recordSchema={sru:recordSchema?}" type="application/atom+xml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		    <param:Parameter name="recordSchema" value="{sru:recordSchema}">      
			      <param:Option label="O&amp;M 1.1" value="om"/>      
			      <param:Option label="O&amp;M 1.0" value="om10"/>      
			      <param:Option label="server-choice" value="server-choice"/>      
		    </param:Parameter>    
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson&amp;parentIdentifier=TropForest&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;subject={dc:subject?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productionStatus={eo:productionStatus?}&amp;illuminationAzimuthAngle={eo:illuminationAzimuthAngle?}&amp;cloudCover={eo:cloudCover?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;illuminationElevationAngle={eo:illuminationElevationAngle?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;classifiedAs={semantic:classifiedAs?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}" type="application/geo+json" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2009-01-01T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2011-08-09T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter maxInclusive="100" minInclusive="0" name="cloudCover" pattern="(\[|\])(100|[0-9]\d?),(100|[0-9]\d?)(\[|\])|(\[|\])?(100|[0-9]\d?)|(100|[0-9]\d?)(\[|\])?|\{(100|[0-9]\d?),(100|[0-9]\d?)\}" value="{eo:cloudCover}"/>    
		
		
		
		
		    <param:Parameter maxInclusive="360" minInclusive="0" name="illuminationAzimuthAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationAzimuthAngle}"/>    
		
		    <param:Parameter maxInclusive="180" minInclusive="0" name="illuminationElevationAngle" pattern="(\[|\])[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?(\[|\])|(\[|\])?[0-9]+(.[0-9]+)?|[0-9]+(.[0-9]+)?(\[|\])?|\{[0-9]+(.[0-9]+)?,[0-9]+(.[0-9]+)?\}" value="{eo:illuminationElevationAngle}"/>    
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
		
										
		
				
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="AVNIR-2" value="AVNIR-2"/>      
      <param:Option label="MSC" value="MSC"/>      
      <param:Option label="SLIM6" value="SLIM6"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="ALOS" value="ALOS"/>      
      <param:Option label="DEIMOS" value="DEIMOS"/>      
      <param:Option label="KOMPSAT" value="KOMPSAT"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="AL1_AV2_2F" value="AL1_AV2_2F"/>      
      <param:Option label="DE1_SL6_2F" value="DE1_SL6_2F"/>      
      <param:Option label="KO2_MSC_2F" value="KO2_MSC_2F"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="1" value="1"/>      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="Astrium Services" value="Astrium Services"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="MS" value="MS"/>      
      <param:Option label="NA" value="NA"/>      
      <param:Option label="OBS" value="OBS"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By illuminationAzimuthAngle (ascending)." value="illuminationAzimuthAngle,eo,1"/>      
      <param:Option label="By illuminationAzimuthAngle (descending)." value="illuminationAzimuthAngle,eo,0"/>      
      <param:Option label="By cloudCover (ascending)." value="cloudCover,eo,1"/>      
      <param:Option label="By cloudCover (descending)." value="cloudCover,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By illuminationElevationAngle (ascending)." value="illuminationElevationAngle,eo,1"/>      
      <param:Option label="By illuminationElevationAngle (descending)." value="illuminationElevationAngle,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Query role="example" time:end="2011-08-09T00:00:00.000Z" time:start="2009-01-01T00:00:00.000Z" xmlns:os="http://a9.com/-/spec/opensearch/1.1/"/>  
  <LongName>Earth Observation Catalogue</LongName>  
  <Image height="64" type="image/png" width="64">https://fedeo.ceos.org/images/esa_favicon.ico</Image>  
  <Image height="16" type="image/vnd.microsoft.icon" width="16">https://fedeo.ceos.org/images/esa_favicon.ico</Image>  
  <Developer>Spacebel s.a.</Developer>  
  <Attribution>Copyright 2017-2020, European Space Agency.</Attribution>  
  <SyndicationRight>open</SyndicationRight>  
  <AdultContent>false</AdultContent>  
  <Language>en-us</Language>  
  <OutputEncoding>UTF-8</OutputEncoding>  
  <InputEncoding>UTF-8</InputEncoding>  
</OpenSearchDescription>
```


**Step 5 - work around (correct OSDD URL)**  
>  The OSDD URL currently returned by IDN/CMR is incorrect as reported Tuesday, April 2, 2024 at 9:58 AM.

In [ ]:
root = ElementTree.fromstring(response.text)
el = root.find('{http://a9.com/-/spec/opensearch/1.1/}title')
print(el.text)

if (el.text == 'OpenSearch Exception'):
    url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=" + short_name
    print("Corrected OSDD URL: " + url_osdd)

In [ ]:
# could not extract this from Atom response due to incorrect link.  Set the expected link
# The correct way to get the OSDD seems using the shortName, not collectionConceptId

# url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=Landsat_8"   # Example 1
# url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?shortName=GEDI_L4A_AGB_Density_V2_1_2056"  # Example 2

In [ ]:
# Get the OSDD document for granule search
response = requests.get( url_osdd )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")

In [ ]:
# Extract URL template for granule search
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
granule_search_template = root.find('os:Url[@rel="results"][@type="application/atom+xml"]', ns).attrib['template']

# collection_template = collection_url_atom.attrib['template']
granule_search_template

https://m2m.cr.usgs.gov/api/open-search/granules.atom?datasetName=LANDSAT_8_C1&timeStart=1972-09-26

In [ ]:
osquerystring = {'time:start': '1972-09-26T00:00:00Z'}
request_url = get_api_request(granule_search_template, osquerystring)

response = requests.get( request_url )

# xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
# md("```xml\n" + xmlstr + "\n```\n")

root = ElementTree.fromstring(response.text)
# Extract <entry> element 
# el = root.find('{http://www.w3.org/2005/Atom}entry')
xmltxt = ElementTree.tostring(root, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

## Obtaining the OpenSearch Description Document (OSDD)

OpenSearch Description Documents (OSDDs) provide necessary information for clients to programmatically formulate valid search requests. Specifically, clients are expected to acquire both the cardinality and the domain of request parameters based on the query template in the OSDD. Collection valids (e.g. spatial bounding coordinates and temporal extent) are also provided in the OSDD in both machine parsable and human readable formats. Collection valids enable clients to formulate valid requests yielding more accurate results. 

The federated system provides both generic and collection specific OSDDs.  IDN provides the generic OSDD to be used for collection search.  Alternatively, clients are also able to fetch a generic OSDD through the CWIC or FedEO OSDD endpoint.  The OSDD request should also include a client identifier string, as recommended by the CEOS OpenSearch Best Practices.

Collection-specific OSDD are returned by IDN as part of collection search results. In a collection specific OSDD, the domain is provided for some parameters (e.g., timeStart and timeEnd) in addition to the request parameter syntax. 

- Generic OSDD request URL example (CMR):  https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=foo  
- Generic OSDD request URL example (FedEO): https://fedeo.ceos.org/opensearch/description.xml 

## Search request

CEOS OpenSearch supports both searching for collections through the IDN and
for granules in a specific collection at one of the data partners (i.e. “CWIC” and “FedEO”). It executes a collection or inventory search, as appropriate, and returns the matching
results.

In order to initialize a valid request, clients will fill request parameters with proper values. The
template of the OpenSearch request is available under the <Url> element in OSDD.

## Search response

OpenSearch does not define or require any specific encoding format for the search response. 

Instead, it defines a set of search-related metadata elements which can be inserted into existing encoding formats. Typically, list-based XML syndication formats - such as RSS 2.0 and Atom 1.0 - are used. 

The OpenSearch response elements include: 

- `totalResults`: number of total results 
- `startIndex`: index number corresponding to the first entry item returned 
- `itemsPerPage`: number of results returned 
- `Query`: search query to get the same search response 

Below is an example of a search response from the IDN in Atom, to the free keyword search “water”. 

In [ ]:
osquerystring = {'count': '1', 'searchTerms': 'water', 'startIndex': '1', 'startPage': '1'}
request_url = get_api_request(collection_template, osquerystring)

response = requests.get( request_url )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
md("```xml\n" + xmlstr + "\n```\n")

Notice the OpenSearch elements which appear inside the feed element. Namespaces referred in the OpenSearch response are listed as follows: 

In [ ]:
import xml.dom.minidom
# Get namespace declarations from root element (all atributes)
n = minidom.parseString(response.text).documentElement
attrs = dict(n.attributes.items())

In [ ]:
# Display namespace declarations as a table
table = pd.DataFrame.from_dict( [ attrs  ] )
table.set_axis([ "URL"], axis=0, inplace=True )
print(table.transpose())

## Error Handling